In [ ]:
import pandas as pd
import os
import neo4j
import numpy as np

# Read books data

In [426]:
df = pd.read_json('goodreads_books_poetry.json', lines=True)

In [427]:
df.head(2)

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
0,,1,[],US,eng,"[{'count': '8', 'name': 'to-read'}, {'count': ...",,false,3.83,,...,11,,1887,https://www.goodreads.com/book/show/16037549-v...,https://images.gr-assets.com/books/1348176637m...,16037549,3,5212748,Vision of Sir Launfal and Other Poems,Vision of Sir Launfal and Other Poems
1,0811223981,2,[],US,,"[{'count': '100', 'name': 'to-read'}, {'count'...",,false,3.83,B00U2WY9U8,...,4,,2015,https://www.goodreads.com/book/show/22466716-f...,https://images.gr-assets.com/books/1404958407m...,22466716,37,41905435,Fairy Tales: Dramolettes,Fairy Tales: Dramolettes


In [428]:
df.columns

Index(['isbn', 'text_reviews_count', 'series', 'country_code', 'language_code',
       'popular_shelves', 'asin', 'is_ebook', 'average_rating', 'kindle_asin',
       'similar_books', 'description', 'format', 'link', 'authors',
       'publisher', 'num_pages', 'publication_day', 'isbn13',
       'publication_month', 'edition_information', 'publication_year', 'url',
       'image_url', 'book_id', 'ratings_count', 'work_id', 'title',
       'title_without_series'],
      dtype='object')

In [429]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36514 entries, 0 to 36513
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   isbn                  36514 non-null  object 
 1   text_reviews_count    36514 non-null  int64  
 2   series                36514 non-null  object 
 3   country_code          36514 non-null  object 
 4   language_code         36514 non-null  object 
 5   popular_shelves       36514 non-null  object 
 6   asin                  36514 non-null  object 
 7   is_ebook              36514 non-null  object 
 8   average_rating        36514 non-null  float64
 9   kindle_asin           36514 non-null  object 
 10  similar_books         36514 non-null  object 
 11  description           36514 non-null  object 
 12  format                36514 non-null  object 
 13  link                  36514 non-null  object 
 14  authors               36514 non-null  object 
 15  publisher          

In [432]:
df = df.replace('',np.nan)

In [434]:
df.drop(columns = ['popular_shelves','isbn','language_code','asin','kindle_asin','edition_information'], inplace = True)

In [458]:
for index, col in enumerate(df.columns):
    print(col, df[col].isna().sum()*100/df.shape[0])

text_reviews_count 0.0
series 0.0
country_code 0.0
is_ebook 0.0
average_rating 0.0
similar_books 0.0
description 0.0
format 0.0
link 0.0
authors 0.0
publisher 0.0
num_pages 0.0
publication_day 0.0
isbn13 0.0
publication_month 0.0
publication_year 0.0
url 0.0
image_url 0.0
book_id 0.0
ratings_count 0.0
work_id 0.0
title 0.0
title_without_series 0.0


In [446]:
df.text_reviews_count.astype('int64')
df.average_rating.astype('float64')
df.num_pages.astype('float64')
df.publication_month.astype('float64')
df.publication_year.astype('float64')

0        1887.0
1        2015.0
2        2008.0
3        1964.0
4        2015.0
          ...  
36509       NaN
36510       NaN
36511       NaN
36512    2013.0
36513    1996.0
Name: publication_year, Length: 36514, dtype: float64

In [457]:
df['format'].fillna(df['format'].value_counts().index[0], inplace = True)
df['publisher'].fillna(df['publisher'].value_counts().index[0], inplace = True)
df['num_pages'].fillna(df['num_pages'].value_counts().index[0], inplace = True)
df['publication_day'].fillna(df['publication_day'].value_counts().index[0], inplace = True)
df['isbn13'].fillna(df['isbn13'].value_counts().index[0], inplace = True)
df['publication_month'].fillna(df['publication_month'].value_counts().index[0], inplace = True)
df['publication_year'].fillna(df['publication_year'].value_counts().index[0], inplace = True)
df['title'].fillna(df['title'].value_counts().index[0], inplace = True)
df['title_without_series'].fillna(df['title_without_series'].value_counts().index[0], inplace = True)
df['description'].fillna(df['description'].value_counts().index[0], inplace = True)

## Extract authors in a dataframe

In [459]:
def extract_author(df, target_col):
    temp_df = pd.DataFrame(columns=['book_id', target_col])
    for index, row in df[['book_id', 'authors']].dropna().iterrows():
        list_of_values = row[1][0]['author_id']
        book_id = row[0]
        #print([list_of_values])
        temp_df = pd.concat([temp_df, pd.DataFrame({'book_id': book_id, target_col: [list_of_values]})])
    return temp_df

In [460]:
authors_df = extract_author(df, 'author')

In [461]:
authors_df.head(2)

,book_id,author
0,16037549,15585
0,22466716,16073


In [462]:
authors_df.to_csv('authors.csv', index = False)

## Extract Publisher

In [464]:
publisher_df = df[['book_id','publisher', 'num_pages', 'publication_day', 'isbn13',
       'publication_month', 'publication_year']]

In [465]:
publisher_df.head(2)

,book_id,publisher,num_pages,publication_day,isbn13,publication_month,publication_year
0,16037549,"Houghton, Mifflin and Company",80,1,9789548689939,11,1887
1,22466716,New Directions,128,20,9780811223980,4,2015


In [467]:
publisher_df = publisher_df[publisher_df.publisher.notna()]

In [468]:
publisher_df.to_csv('publisher.csv', index = False)

## Extract similar books

In [469]:
def extract_similarBooks(df, target_col):
    temp_df = pd.DataFrame(columns=['book_id', target_col])
    for index, row in df[['book_id', 'similar_books']].dropna().iterrows():
        list_of_values = row[1]
        book_id = row[0]
        temp_df = pd.concat([temp_df, pd.DataFrame({'book_id': book_id, target_col: list_of_values})])
    return temp_df

In [472]:
similarBooks_df = extract_similarBooks(df,'similar_books')

KeyboardInterrupt: 

In [471]:
similarBooks_df.head(2)

,book_id,similar_books
0,926667,1230072
1,926667,315167


In [ ]:
similarBooks_df.to_csv('similarBooks.csv', index = False)

## Extract series

In [ ]:
def extract_series(df, target_col):
    temp_df = pd.DataFrame(columns=['book_id', target_col])
    for index, row in df[['book_id', 'series']].dropna().iterrows():
        list_of_values = row[1]
        book_id = row[0]
        temp_df = pd.concat([temp_df, pd.DataFrame({'book_id': book_id, target_col: list_of_values})])
    return temp_df

In [ ]:
series_df = extract_series(df, 'series')

In [ ]:
series_df.head(2)

In [ ]:
series_df.to_csv('bookSeries.csv', index = False)

## Extract books alone

In [474]:
df.columns

Index(['text_reviews_count', 'series', 'country_code', 'is_ebook',
       'average_rating', 'similar_books', 'description', 'format', 'link',
       'authors', 'publisher', 'num_pages', 'publication_day', 'isbn13',
       'publication_month', 'publication_year', 'url', 'image_url', 'book_id',
       'ratings_count', 'work_id', 'title', 'title_without_series'],
      dtype='object')

In [475]:
books_df = df[['book_id', 'text_reviews_count', 'country_code',
       'is_ebook', 'average_rating', 'description', 'format', 'link', 'url',
       'image_url', 'ratings_count', 'work_id', 'title',
       'title_without_series']]

In [476]:
books_df.head(2)

,book_id,text_reviews_count,country_code,is_ebook,average_rating,description,format,link,url,image_url,ratings_count,work_id,title,title_without_series
0,16037549,1,US,false,3.83,Number 30 in a series of literary pamphlets pu...,Paperback,https://www.goodreads.com/book/show/16037549-v...,https://www.goodreads.com/book/show/16037549-v...,https://images.gr-assets.com/books/1348176637m...,3,5212748,Vision of Sir Launfal and Other Poems,Vision of Sir Launfal and Other Poems
1,22466716,2,US,false,3.83,Fairy Tales gathers the unconventional verse d...,Paperback,https://www.goodreads.com/book/show/22466716-f...,https://www.goodreads.com/book/show/22466716-f...,https://images.gr-assets.com/books/1404958407m...,37,41905435,Fairy Tales: Dramolettes,Fairy Tales: Dramolettes


In [477]:
ebooks_df = books_df[books_df.is_ebook == 'true']

In [478]:
not_ebooks_df = books_df[books_df.is_ebook == 'false']

In [503]:
ebooks_df.to_csv('ebooks.csv', index = False)
#not_ebooks_df.to_csv('not_ebooks.csv', index = False)

In [480]:
not_ebooks = not_ebooks_df.copy()

In [481]:
not_ebooks.description = not_ebooks.description.apply(lambda x: x.replace('"', "'"))
not_ebooks.title = not_ebooks.title.apply(lambda x: x.replace('"', "'"))
not_ebooks.title_without_series = not_ebooks.title_without_series.apply(lambda x: x.replace('"', "'"))

In [482]:
not_ebooks.to_csv('not_ebooks.csv', index = False)

In [483]:
not_ebooks[not_ebooks.apply(lambda row: row.astype(str).str.contains("\"").any(), axis=1)]

,book_id,text_reviews_count,country_code,is_ebook,average_rating,description,format,link,url,image_url,ratings_count,work_id,title,title_without_series


# Read reviews data

In [485]:
df_reviews = pd.read_json('goodreads_reviews_poetry.json', lines=True)

In [486]:
df_reviews.columns

Index(['user_id', 'book_id', 'review_id', 'rating', 'review_text',
       'date_added', 'date_updated', 'read_at', 'started_at', 'n_votes',
       'n_comments'],
      dtype='object')

In [487]:
df_reviews.head(3)

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,3ca7375dba942a760e53b726c472a7dd,402128,28423ff309bc896c071a8d9df4a10e8a,5,I have three younger siblings and we grew up w...,Tue Jun 12 08:59:04 -0700 2012,Fri Jun 15 11:41:12 -0700 2012,,,0,0
1,0ef32090550901ead25cb0ea21c4d36b,92270,2db1180992e2b0b1631a3ac5644bde84,5,This is my favorite collection of poetry.,Mon Apr 14 18:42:40 -0700 2014,Mon Apr 14 18:43:05 -0700 2014,Wed Jan 01 00:00:00 -0800 1997,,0,0
2,0ef32090550901ead25cb0ea21c4d36b,908708,bca57fa40e92c9261b00b03dbebd96fe,4,"He's so disturbing. So very, very disturbing.",Tue Apr 22 13:58:10 -0700 2008,Tue Apr 22 13:58:33 -0700 2008,,,0,0


In [488]:
df_reviews[df_reviews.apply(lambda row: row.astype(str).str.contains("\"").any(), axis=1)]

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
12,d0c8e4089c1435ff9e1ab3f2bc5f88ff,5885639,73acf3ef1555145e264933740c43f5d0,4,k`d@ frwq jwyd@ y'sr lqr bqSy'dh l`dhb@ lmlyy'...,Sat Dec 21 01:53:55 -0800 2013,Sun Dec 22 00:25:03 -0800 2013,Fri Dec 20 00:00:00 -0800 2013,,2,0
13,d0c8e4089c1435ff9e1ab3f2bc5f88ff,5948950,23a0016b3f4675a69e64e84a19b27359,5,"jmyl jd [?][?] \n "" lmdh 'rki `l~ klW shy bqy ...",Sat Dec 21 00:33:02 -0800 2013,Sun Dec 22 00:25:23 -0800 2013,Fri Dec 20 00:00:00 -0800 2013,,0,0
15,f88032f4ad97b46654fe59ce3387cf5d,23668465,ab273055bf0e8defc685e1cea51899c2,4,Beyond the Words is an anthology of poetry and...,Thu Jul 23 17:25:59 -0700 2015,Sat Oct 17 00:19:55 -0700 2015,Thu Oct 01 00:00:00 -0700 2015,Thu Jul 23 00:00:00 -0700 2015,1,0
16,f88032f4ad97b46654fe59ce3387cf5d,18600808,4415c79a0d78929a39018111d4a8565a,0,Thoughts of a Pure Mind by Calvin Bland is a c...,Sun Mar 23 06:32:03 -0700 2014,Sun Mar 23 06:35:12 -0700 2014,Tue Jan 01 00:00:00 -0800 2013,,1,0
17,f88032f4ad97b46654fe59ce3387cf5d,18745066,a99dcdeea0a543cffda08f7ee20bfcae,4,Seeing Past Sickness is a collection of 17 pie...,Mon Nov 04 15:37:37 -0800 2013,Tue Dec 30 12:25:07 -0800 2014,Thu Jul 03 00:00:00 -0700 2014,Sat Jun 28 00:00:00 -0700 2014,1,0
...,...,...,...,...,...,...,...,...,...,...,...
154538,9050eb3c9d81eac0194e2e882f15da19,50488,901a3ea8bb6ccac83c461f60e402e8db,4,"""His name is Boredom. He sniffs, wipes his eye...",Fri Mar 27 20:35:57 -0700 2015,Fri Mar 27 20:38:09 -0700 2015,,,1,0
154545,854c10d0e84107c80db6dd614b12e020,25882202,c8a19a2df288b8f50fd3d57ccf87dd7c,4,z chshm tw kh dr bnd~ drwysh \n jhn~ zndn~ st ...,Fri Sep 18 14:55:29 -0700 2015,Fri Sep 18 15:08:12 -0700 2015,,,1,0
154546,854c10d0e84107c80db6dd614b12e020,25099715,c6f05e0d7ee3f7f8ad5320dcc624a486,5,"""`mqi akhryn Hrf h \n mthli ystdn knri drh ~ \...",Fri Apr 03 07:07:04 -0700 2015,Fri Apr 03 07:11:55 -0700 2015,,,5,0
154553,8ba77e3c745ebddccc6306fc3c6bb25e,9404584,844d9609e96ec611166ef20bf270233f,5,Emily Dickinson left a large cache of poetry -...,Tue Sep 04 14:58:29 -0700 2012,Tue Dec 24 03:26:39 -0800 2013,Fri Sep 21 10:09:53 -0700 2012,Tue Sep 04 14:58:29 -0700 2012,4,1


In [489]:
df_reviews.review_text = df_reviews.review_text.apply(lambda x: x.replace('"', "'"))
# df_reviews.review_text = df_reviews.review_text.apply(lambda x: x.replace('"', "'"))

In [496]:
df_reviews.isnull().any()

user_id         False
book_id         False
review_id       False
rating          False
review_text     False
date_added      False
date_updated    False
read_at         False
started_at      False
n_votes         False
n_comments      False
dtype: bool

In [497]:
df_reviews = df_reviews[['user_id', 'book_id', 'review_id', 'rating']]

In [499]:
df_reviews.rating.value_counts()

5    58510
4    47266
3    28028
2     9942
0     6732
1     4077
Name: rating, dtype: int64

In [500]:
df_reviews.to_csv('reviews.csv', index = False)

In [502]:
print("number of users - ",len(set(df_reviews.user_id)))
print("number of books - ", len(set(df.book_id)))
print("number of authors - ", len(set(authors_df.author)))
print("number of publishers - ", len(set(df.publisher)))
print("number of series - ", len(set(series_df.series)))
print("number of reviews - ",len(set(df_reviews.review_id)))

number of users -  47400
number of books -  36514
number of authors -  12165
number of publishers -  8185
number of series -  613
number of reviews -  154555
